In [16]:
# This notebook will take in a directory containing directories named after each particpant, 
# and in each one of those, contain 6 folders named after the gesture classes, 
# and in each one of those, contain a number of .mp4 videos.

# for each video, it will extract the partipant name, the gesture name, and a video index pertaining to that gesture
# and then it will build a directory that contains 6 directories for each gesture class.
# Inside each gesture directory a folder named in the following convention:
# <participant_name>_<gesture_class>_<gesture_video_index_num>
# will be created for each video.
# Inside each of those directories will be a series of .jpgs pertaining to the frames extracted from the video.

In [65]:
import os
from pathlib import Path
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

import time

from threading import Thread
import sys

from queue import Queue

In [6]:
GESTURE_NAMES = ["CLOCKWISE", "COUNTERCLOCKWISE", "DOWN", "UP", "LEFT", "RIGHT"]

In [10]:
VIDEOS_DIR = Path("D:/__School/__Masters/____2021fALL/5280_aiwearables/gesture_videos_2nd_iter")

In [52]:
TARGET_DIR = Path("D:/__School/__Masters/____2021fALL/5280_aiwearables/gesture_frames_2nd_iter")

In [25]:
def get_dir_names_all_videos(home_dir):
    """returns the names in a directories subfolders"""
    video_fnames = []
    for path, subdirs, files in os.walk(home_dir):
        for name in files:
            video_fnames.append(str(Path(os.path.join(path, name))))
    return video_fnames

In [26]:
video_dir_names = get_dir_names_all_videos(VIDEOS_DIR)

In [27]:
len(video_dir_names)

350

In [42]:
video_dir_names[0]

'D:\\__School\\__Masters\\____2021fALL\\5280_aiwearables\\gesture_videos_2nd_iter\\cooper_s\\CLOCKWISE\\WIN_20211021_13_04_32_Pro.mp4'

In [31]:
def get_metadata_all_videos(video_dir_names):
    fps_length_data = []
    
    for i, vid_dir in enumerate(video_dir_names):
        if i % 100 == 0:
            print(f"processing {i} of {len(video_dir_names)}...")
        split_dir = vid_dir.split("\\")
        fname = split_dir[-1]
        gesture_name = split_dir[-2].replace(" ", "").upper()
        subject_initials = split_dir[-3]
        
        video = cv2.VideoCapture(str(vid_dir))
        fps = video.get(cv2.CAP_PROP_FPS)
        frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        duration_seconds = frame_count / fps
        
        fps_length_data.append({"file_name": fname,
                                "subject_initials": subject_initials,
                                "gesture_name": gesture_name,
                                "frames_per_second": fps,
                                "frame_count": frame_count,
                                "duration_seconds": duration_seconds,
                                "dir_name": vid_dir})
    df = pd.DataFrame(fps_length_data)
    print("Done!")
    return df

In [32]:
df = get_metadata_all_videos(video_dir_names)

processing 0 of 350...
processing 100 of 350...
processing 200 of 350...
processing 300 of 350...
Done!


In [33]:
df.head(3)

,file_name,subject_initials,gesture_name,frames_per_second,frame_count,duration_seconds,dir_name
0,WIN_20211021_13_04_32_Pro.mp4,cooper_s,CLOCKWISE,10.261673,34,3.313300,D:\__School\__Masters\____2021fALL\5280_aiwear...
1,WIN_20211021_13_04_38_Pro.mp4,cooper_s,CLOCKWISE,10.256942,33,3.217333,D:\__School\__Masters\____2021fALL\5280_aiwear...
2,WIN_20211021_13_04_44_Pro.mp4,cooper_s,CLOCKWISE,10.256942,33,3.217333,D:\__School\__Masters\____2021fALL\5280_aiwear...


In [43]:
df["rounded_fps"] = df.frames_per_second.round()

In [45]:
df.rounded_fps.value_counts()

30.0    300
10.0     50
Name: rounded_fps, dtype: int64

In [46]:
df.subject_initials.value_counts()

cooper_s      50
Daniel_M      50
emmanuel_z    50
Ian_z         50
Lori_L        50
Madeline_U    50
Miguel_Q      50
Name: subject_initials, dtype: int64

In [47]:
df.gesture_name.value_counts()

DOWN                70
LEFT                70
RIGHT               70
UP                  70
CLOCKWISE           35
COUNTERCLOCKWISE    35
Name: gesture_name, dtype: int64

In [51]:
df.duration_seconds.round().value_counts()

3.0    173
2.0    150
4.0     14
1.0     13
Name: duration_seconds, dtype: int64

In [56]:
def create_video_folder_names(df, gesture_names):
    """returns a dataframe with a new column formatted 
    <subject_initials>_<gesture_name>_<incremented_id>
    based on gesture classes"""
    df_list = []
    
    def for_each_row(row):
        # build video filename
        folder_name = f"{row.subject_initials}_{row.gesture_name}_{row.increment_id:05d}"
        return folder_name
    
    for gesture_name in gesture_names:
         # creates a new dataframe with increment values for each gesture
        dff = df[df["gesture_name"] == gesture_name].copy()
        dff["increment_id"] = np.arange(1, dff.shape[0]+1)
        dff["gesture_video_foldername"] = dff.apply(for_each_row, axis=1)
        df_list.append(dff)
    
        
    dff = pd.concat(df_list)
    dff.reset_index(drop=True, inplace=True)
    return dff

In [57]:
df = create_video_folder_names(df, GESTURE_NAMES)

In [59]:
def build_frames_directory(df, target_directory, gesture_names):
    """builds a directory for each class, and in each class a folder name"""
    
    def for_each_row(row):
        # make internal video directories
        gesture_dir = Path(target_directory).joinpath(row.gesture_name)
        video_dir = gesture_dir.joinpath(row.gesture_video_foldername)
        video_dir.mkdir(parents=True, exist_ok=True)
        
    for gesture_name in gesture_names:
        # make class directories
        gesture_directory = Path(target_directory).joinpath(gesture_name)
        gesture_directory.mkdir(parents=True, exist_ok=True)
    
    df.apply(for_each_row, axis=1)

In [60]:
build_frames_directory(df, TARGET_DIR, GESTURE_NAMES)

In [ ]:
def video_to_frames(input_dir, output_dir):
    """take in a video from input_dir and output the frames to the output_dir"""
    video = cv2.VideoCapture(str(input_dir))
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(0, frame_count):
        success, image = video.read()
        if not success:
            continue
        cv2.imwrite( str(output_dir) + f"\\img_{i+1:05d}.jpg", image)     # save frame as JPEG file

In [76]:
# this cell will implement all the code necessary for the 50% faster image reading and saving

class FileVideoStream:
    def __init__(self, path, queueSize=160):
        # initialize the file video stream along with the boolean
        # used to indicate if the thread should be stopped or not
        self.stream = cv2.VideoCapture(path)
        self.stopped = False
        # initialize the queue used to store frames read from
        # the video file
        self.Q = Queue(maxsize=queueSize)
        
    def start(self):
        # start a thread to read frames from the file video stream
        t = Thread(target=self.update, args=())
        t.daemon = True
        t.start()
        return self
    
    def update(self):
        # keep looping infinitely
        while True:
            # if the thread indicator variable is set, stop the
            # thread
            if self.stopped:
                return
            # otherwise, ensure the queue has room in it
            if not self.Q.full():
                # read the next frame from the file
                (grabbed, frame) = self.stream.read()
                # if the `grabbed` boolean is `False`, then we have
                # reached the end of the video file
                if not grabbed:
                    self.stop()
                    return
                # add the frame to the queue
                self.Q.put(frame)
                
    def read(self):
        # return next frame in the queue
        return self.Q.get()
    
    def more(self):
        # return True if there are still frames in the queue
        return self.Q.qsize() > 0
    
    def stop(self):
        # indicate that the thread should be stopped
        self.stopped = True

In [92]:
def process_video_to_frames(input_file, output_dir, x, y):

    fvs = FileVideoStream(input_file).start()
    time.sleep(1.0)
    
    frame_count = 0
    while fvs.more():
        frame_count += 1
        # grab the frame from the threaded video file stream, resize
        # it, and convert it to grayscale (while still retaining 3
        # channels)
        frame = fvs.read()
        frame = cv2.resize(frame, (x,y))
        
        # save frame to directory
        cv2.imwrite( str(output_dir) + f"\\img_{frame_count:05d}.jpg", frame)
        
        cv2.waitKey(1)
        
    cv2.destroyAllWindows()
    fvs.stop()

In [93]:
def populate_frames(df, target_directory, gesture_names):
    """populate each of the video subfolders with individual frames from the videos"""
    def per_row(row):
        gesture_dir = Path(target_directory).joinpath(row.gesture_name)
        output_video_dir = gesture_dir.joinpath(row.gesture_video_foldername)
        input_video_dir = row.dir_name
        print(f"Processing {row.gesture_video_foldername}")
        process_video_to_frames(input_video_dir, output_video_dir, 320, 240)
    df.apply(per_row, axis=1)

In [94]:
populate_frames(df, TARGET_DIR, GESTURE_NAMES)

Processing cooper_s_CLOCKWISE_00001
Processing cooper_s_CLOCKWISE_00002
Processing cooper_s_CLOCKWISE_00003
Processing cooper_s_CLOCKWISE_00004
Processing cooper_s_CLOCKWISE_00005
Processing Daniel_M_CLOCKWISE_00006
Processing Daniel_M_CLOCKWISE_00007
Processing Daniel_M_CLOCKWISE_00008
Processing Daniel_M_CLOCKWISE_00009
Processing Daniel_M_CLOCKWISE_00010
Processing emmanuel_z_CLOCKWISE_00011
Processing emmanuel_z_CLOCKWISE_00012
Processing emmanuel_z_CLOCKWISE_00013
Processing emmanuel_z_CLOCKWISE_00014
Processing emmanuel_z_CLOCKWISE_00015
Processing Ian_z_CLOCKWISE_00016
Processing Ian_z_CLOCKWISE_00017
Processing Ian_z_CLOCKWISE_00018
Processing Ian_z_CLOCKWISE_00019
Processing Ian_z_CLOCKWISE_00020
Processing Lori_L_CLOCKWISE_00021
Processing Lori_L_CLOCKWISE_00022
Processing Lori_L_CLOCKWISE_00023
Processing Lori_L_CLOCKWISE_00024
Processing Lori_L_CLOCKWISE_00025
Processing Madeline_U_CLOCKWISE_00026
Processing Madeline_U_CLOCKWISE_00027
Processing Madeline_U_CLOCKWISE_00028
P

Processing Lori_L_LEFT_00042
Processing Lori_L_LEFT_00043
Processing Lori_L_LEFT_00044
Processing Lori_L_LEFT_00045
Processing Lori_L_LEFT_00046
Processing Lori_L_LEFT_00047
Processing Lori_L_LEFT_00048
Processing Lori_L_LEFT_00049
Processing Lori_L_LEFT_00050
Processing Madeline_U_LEFT_00051
Processing Madeline_U_LEFT_00052
Processing Madeline_U_LEFT_00053
Processing Madeline_U_LEFT_00054
Processing Madeline_U_LEFT_00055
Processing Madeline_U_LEFT_00056
Processing Madeline_U_LEFT_00057
Processing Madeline_U_LEFT_00058
Processing Madeline_U_LEFT_00059
Processing Madeline_U_LEFT_00060
Processing Miguel_Q_LEFT_00061
Processing Miguel_Q_LEFT_00062
Processing Miguel_Q_LEFT_00063
Processing Miguel_Q_LEFT_00064
Processing Miguel_Q_LEFT_00065
Processing Miguel_Q_LEFT_00066
Processing Miguel_Q_LEFT_00067
Processing Miguel_Q_LEFT_00068
Processing Miguel_Q_LEFT_00069
Processing Miguel_Q_LEFT_00070
Processing cooper_s_RIGHT_00001
Processing cooper_s_RIGHT_00002
Processing cooper_s_RIGHT_00003
Pro